In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: d:\Studied\Scalable Machine Learning\ID2223_LAB1_AirQuality
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-14 00:38:01,400 INFO: Initializing external client
2025-11-14 00:38:01,401 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-14 00:38:02,969 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1271980


'{"country": "china", "city": "beijing", "street": "dongchengdongsi", "aqicn_url": "https://api.waqi.info/feed/@446", "latitude": "39.929", "longitude": "116.417"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,55.0,china,beijing,dongchengdongsi,2025-11-14,https://api.waqi.info/feed/@446


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 40.0°N 116.5°E
Elevation 48.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-13,9.40,0.0,9.028754,203.498581,beijing
1,2025-11-14,9.15,0.0,7.289445,212.905243,beijing
2,2025-11-15,10.30,0.0,11.753877,207.349792,beijing
3,2025-11-16,0.75,0.0,10.182337,315.000092,beijing
4,2025-11-17,0.90,0.0,10.691453,315.000092,beijing
5,2025-11-18,2.60,0.0,3.319036,282.528809,beijing
6,2025-11-19,3.35,0.0,6.034700,287.354095,beijing


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-14 00:47:01,527 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271980/fs/1265734/fg/1637760


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271980/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735264
         }
       },
       "result": {
         "observed_value": 55.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-13T11:47:01.000527Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectat

In [11]:
air_quality_fg.read().sort_values("date", ascending=False).head(10)   # 看最近 10 天

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.48s) 


,date,pm25,country,city,street,url
4252,2025-11-14 00:00:00+00:00,55.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4251,2025-11-13 00:00:00+00:00,21.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4250,2025-11-12 00:00:00+00:00,171.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4249,2025-11-11 00:00:00+00:00,99.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4248,2025-11-10 00:00:00+00:00,57.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4247,2025-11-09 00:00:00+00:00,21.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4246,2025-11-08 00:00:00+00:00,137.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4245,2025-11-07 00:00:00+00:00,167.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4244,2025-11-06 00:00:00+00:00,194.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4243,2025-11-05 00:00:00+00:00,195.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446


In [12]:
# FOR C GRADE

# compute lags using strictly previous days from v1, then insert today's row into v2 
air_quality_fg_v2 = fs.get_feature_group(name='air_quality', version=2)

# complete the columns
today_core = aq_today_df[['date', 'pm25']].copy()
today_core['pm25'] = today_core['pm25'].astype('float32')
today_core['country'] = country
today_core['city'] = city
today_core['street'] = street
today_core['url'] = aqicn_url

# convert to UTC+0
today_core['date'] = pd.to_datetime(today_core['date'], utc=True).dt.tz_convert(None)
today_ts = pd.to_datetime(today)  

# we use the historical values before today to be the lagged values for PM2.5
hist_v1 = air_quality_fg.read()[['date','country','city','street','pm25']].copy()
hist_v1['date'] = pd.to_datetime(hist_v1['date'], utc=True).dt.tz_convert(None)
hist_v1 = hist_v1[
    (hist_v1['country'] == country) &
    (hist_v1['city'] == city) &
    (hist_v1['street'] == street) &
    (hist_v1['date'] < today_ts)           
].sort_values('date')

# DROP duplicates
hist_v1 = hist_v1.drop_duplicates(subset=['date'], keep='last')

# the last three will be yesterday, the day before yesterday and the day before the day before yesterday
last3 = hist_v1.tail(3)['pm25'].tolist()

if len(last3) == 3:
    today_row = today_core.copy()
    today_row['pm25_lag1'] = last3[-1]
    today_row['pm25_lag2'] = last3[-2]
    today_row['pm25_lag3'] = last3[-3]

    today_row[['pm25', 'pm25_lag1', 'pm25_lag2', 'pm25_lag3']] = \
        today_row[['pm25', 'pm25_lag1', 'pm25_lag2', 'pm25_lag3']].astype('float32')

    # align with the schema of v2 table
    fg_cols = [f.name for f in air_quality_fg_v2.features]
    for c in fg_cols:
        if c not in today_row.columns:
            today_row[c] = None
    today_row = today_row[fg_cols]

    air_quality_fg_v2.insert(today_row, wait=True)
    print(f"[air_quality v2] inserted {today_ts.date()} with lag1/2/3.")
else:
    print("[air_quality v2] Not enough strictly previous-day history to compute all three lags. Skipped.")




Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.98s) 
2025-11-14 00:54:09,090 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271980/fs/1265734/fg/1596002


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271980/jobs/named/air_quality_2_offline_fg_materialization/executions
2025-11-14 00:54:26,187 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-14 00:54:32,725 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-14 00:56:16,083 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-14 00:56:16,234 INFO: Waiting for log aggregation to finish.
2025-11-14 00:56:31,648 INFO: Execution finished successfully.
[air_quality v2] inserted 2025-11-14 with lag1/2/3.


In [13]:
air_quality_fg_v2.read().sort_values("date", ascending=False).head(10)   # 看最近 10 天


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.55s) 


,date,pm25,country,city,street,url,pm25_lag1,pm25_lag2,pm25_lag3
4249,2025-11-14 00:00:00+00:00,55.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,21.0,171.0,99.0
4248,2025-11-13 00:00:00+00:00,21.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,171.0,99.0,57.0
4247,2025-11-12 00:00:00+00:00,171.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,99.0,57.0,21.0
4246,2025-11-11 00:00:00+00:00,99.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,57.0,21.0,137.0
4245,2025-11-10 00:00:00+00:00,57.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,21.0,137.0,129.0
4244,2025-11-09 00:00:00+00:00,21.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,137.0,129.0,194.0
4243,2025-11-08 00:00:00+00:00,137.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,129.0,194.0,195.0
4241,2025-11-07 00:00:00+00:00,167.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,194.0,195.0,142.0
4240,2025-11-06 00:00:00+00:00,194.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,195.0,142.0,108.0
4239,2025-11-05 00:00:00+00:00,195.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,142.0,108.0,45.0


In [14]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-14 00:57:10,327 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271980/fs/1265734/fg/1595945


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271980/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-14 00:57:27,595 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-14 00:57:34,095 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-14 00:59:15,485 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-14 00:59:15,630 INFO: Waiting for log aggregation to finish.
2025-11-14 00:59:27,730 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733212
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-13T11:57:10.000327Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
